# Airline on Time Data
## Introduction
Have you ever been stuck in an airport because your flight was delayed or cancelled and wondered if you could have predicted it if you'd had more data? This is your chance to find out. 

The data: The data consists of flight arrival and departure details for all commercial flights within the USA, from October 1987 to April 2008. This is a large dataset: there are nearly 120 million records in total, and takes up 1.6 gigabytes of space compressed and 12 gigab ytes when uncompressed. The data comes originally from RITA where it is described in detail. You can download the data [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7).

**However only data for 2007 will be used for this project**

Variable descriptions: 

S/N | Name | Description 
-- | -- | --
1 | `Year` | 1987-2008 (2007 for this analysis)
2 | `Month` | 1-12 
3 | `DayofMonth` | 1-31 
4 | `DayOfWeek` | 1 (Monday) - 7 (Sunday) 
5 | `DepTime` | actual departure time (local, hhmm) 
6 | `CRSDepTime` | scheduled departure time (local, hhmm) 
7 | `ArrTime` | actual arrival time (local, hhmm) 
8 | `CRSArrTime` | scheduled arrival time (local, hhmm) 
9 | `UniqueCarrier` | unique carrier code 
10 | `FlightNum` | flight number 
11 | `TailNum` | plane tail number 
12 | `ActualElapsedTime` | actual elapsed time in minutes 
13 | `CRSElapsedTime` | scheduled elapsed time in minutes 
14 | `AirTime` | the time from the moment an aircraft leaves the surface until it comes into contact with the surface at the next point of landing;,in minutes 
15 | `ArrDelay` | arrival delay, in minutes 
16 | `DepDelay` | departure delay, in minutes 
17 | `Origin` | origin, IATA airport code 
18 | `Dest` | destination, IATA airport code 
19 | `Distance` | distance covered, in miles 
20 | `TaxiIn` | taxi in time, in minutes 
21 | `TaxiOut` | taxi out time in minutes 
22 | `Cancelled` | was the flight cancelled? 
23 | `CancellationCode` | reason for cancellation (A = carrier, B = weather, C = NAS, D = security) 
24 | `Diverted` | 1 = yes, 0 = no 
25 | `CarrierDelay` | delay within air carrier's control, in minutes 
26 | `WeatherDelay` | delay caused by extreme weather conditions, in minutes 
27 | `NASDelay` | delay within the NAS control, in minutes 
28 | `SecurityDelay` | delay due to security checks, breach or faulty security equipment, in minutes 
29 | `LateAircraftDelay` | delay due to the late arrival of the same aircraft at a previous airport, in minutes

* The International Air Transport Association's (IATA) Location Identifier is a unique 3-letter code (also commonly known as IATA code) used in aviation and also in logistics to identify an airport.
* A computer reservation system or a central reservation system (CRS) is a web-based software used by travel agencies and travel management companies to retrieve and conduct transactions related to air travel, hotels, car rental, or other activities. 

**Questions of Interest for the 2007 data**
* When is the best time of day/day of week/time of year to fly to minimise delays?
* Do older planes suffer more delays?
* How does the number of people flying between different locations change over time?
* How well does weather predict plane delays?
* Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?

In [68]:
# import modules and libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max.columns', 500)
plt.style.use('seaborn-dark-palette')
%matplotlib inline

## Data Wrangling
### Gathering data

In [53]:
airline_df = pd.read_csv('2007.csv')
airports = pd.read_csv('airports.csv')
carriers = pd.read_csv('carriers.csv')
plane_data = pd.read_csv('plane-data.csv')

In [54]:
airline_df.shape

(7453215, 29)

It's a reasonably sized data with 7.4 million entries and 29 variables

### Assessing the data

In [55]:
airline_df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2007,1,1,1,1232.0,1225,1341.0,1340,WN,2891,N351,69.0,75.0,54.0,1.0,7.0,SMF,ONT,389,4,11,0,NaN,0,0,0,0,0,0
1,2007,1,1,1,1918.0,1905,2043.0,2035,WN,462,N370,85.0,90.0,74.0,8.0,13.0,SMF,PDX,479,5,6,0,NaN,0,0,0,0,0,0
2,2007,1,1,1,2206.0,2130,2334.0,2300,WN,1229,N685,88.0,90.0,73.0,34.0,36.0,SMF,PDX,479,6,9,0,NaN,0,3,0,0,0,31
3,2007,1,1,1,1230.0,1200,1356.0,1330,WN,1355,N364,86.0,90.0,75.0,26.0,30.0,SMF,PDX,479,3,8,0,NaN,0,23,0,0,0,3
4,2007,1,1,1,831.0,830,957.0,1000,WN,2278,N480,86.0,90.0,74.0,-3.0,1.0,SMF,PDX,479,3,9,0,NaN,0,0,0,0,0,0


In [67]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7453215 entries, 0 to 7453214
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             int64  
 20  TaxiOut            int64  
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       int64  
 25  WeatherDelay      

Percentage of missing values per column

In [8]:
percentNaN = airline_df.isnull().sum()/len(airline_df) * 100
percentNaN

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               2.156761
CRSDepTime            0.000000
ArrTime               2.387252
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               0.000295
ActualElapsedTime     2.387252
CRSElapsedTime        0.013337
AirTime               2.387252
ArrDelay              2.387252
DepDelay              2.156761
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                0.000000
TaxiOut               0.000000
Cancelled             0.000000
CancellationCode     97.843226
Diverted              0.000000
CarrierDelay          0.000000
WeatherDelay          0.000000
NASDelay              0.000000
SecurityDelay         0.000000
LateAircraftDelay     0.000000
dtype: float64

`CancellationCode` seems to have the highest percentage of NaN values but that's expected because it shows that a huge majority of the flights weren't cancelled.

In [9]:
percentNaN[percentNaN != 0]

DepTime               2.156761
ArrTime               2.387252
TailNum               0.000295
ActualElapsedTime     2.387252
CRSElapsedTime        0.013337
AirTime               2.387252
ArrDelay              2.387252
DepDelay              2.156761
CancellationCode     97.843226
dtype: float64

Let's look at the data for the above variables excluding `CancellationCode`

In [57]:
airline_df[airline_df.DepTime.isnull()].sample(15)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
5774048,2007,10,19,5,NaN,1920,NaN,2010,OH,5433,0,NaN,50.0,NaN,NaN,NaN,CMH,CVG,116,0,0,1,B,0,0,0,0,0,0
317023,2007,1,22,1,NaN,1929,NaN,2045,DL,1967,N912DE,NaN,76.0,NaN,NaN,NaN,LGA,DCA,214,0,0,1,C,0,0,0,0,0,0
2134647,2007,4,16,1,NaN,1930,NaN,2100,DL,1933,N915DE,NaN,90.0,NaN,NaN,NaN,BOS,LGA,185,0,0,1,C,0,0,0,0,0,0
149630,2007,1,25,4,NaN,1420,NaN,1610,YV,7344,N855MJ,NaN,110.0,NaN,NaN,NaN,MHT,IAD,418,0,0,1,A,0,0,0,0,0,0
1620348,2007,3,2,5,NaN,1720,NaN,1935,MQ,4362,0,NaN,75.0,NaN,NaN,NaN,ORD,DTW,235,0,0,1,B,0,0,0,0,0,0
7027125,2007,12,11,2,NaN,1130,NaN,1320,OO,2931,0,NaN,110.0,NaN,NaN,NaN,AUS,MCI,650,0,0,1,B,0,0,0,0,0,0
3168405,2007,6,25,1,NaN,1510,NaN,1640,XE,2372,0,NaN,90.0,NaN,NaN,NaN,GPT,IAH,376,0,0,1,B,0,0,0,0,0,0
1086489,2007,2,24,6,NaN,840,NaN,1320,AA,1120,0,NaN,160.0,NaN,NaN,NaN,LAS,DFW,1055,0,0,1,B,0,0,0,0,0,0
2777196,2007,5,21,1,NaN,800,NaN,1113,EV,4197,N858AS,NaN,133.0,NaN,NaN,NaN,DSM,ATL,743,0,0,1,A,0,0,0,0,0,0
146739,2007,1,22,1,NaN,715,NaN,908,YV,7056,N445YV,NaN,113.0,NaN,NaN,NaN,COD,DEN,392,0,0,1,B,0,0,0,0,0,0


I suspect that most missing values in the other columns are as a result of the flight being cancelled and as such, the flight can't have an arrival time or departure time. Let's look for missing data in the flights that weren't cancelled.

In [11]:
flights_NotCancelled = airline_df.query('Cancelled == 0')

In [12]:
percentNaN = flights_NotCancelled.isnull().sum()/len(airline_df) * 100
percentNaN[percentNaN != 0]

ArrTime               0.230491
ActualElapsedTime     0.230491
CRSElapsedTime        0.009754
AirTime               0.230491
ArrDelay              0.230491
CancellationCode     97.843226
dtype: float64

Notice that it's just an insignificant amount that contains NaN values. Let's have a look at it.

In [13]:
# selecting the missing values in the flights that weren't cancelled
# dropping the CancellationCode column first
flights_NotCancelled[flights_NotCancelled.drop('CancellationCode', axis=1).ArrDelay.isnull()]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
3340,2007,1,2,2,802.0,755,NaN,1115,WN,837,...,0,8,0,NaN,1,0,0,0,0,0
8000,2007,1,4,4,1627.0,1610,NaN,1735,WN,2474,...,0,15,0,NaN,1,0,0,0,0,0
8014,2007,1,4,4,1951.0,1905,NaN,2005,WN,2860,...,0,9,0,NaN,1,0,0,0,0,0
8447,2007,1,4,4,845.0,845,NaN,1245,WN,76,...,0,7,0,NaN,1,0,0,0,0,0
8827,2007,1,4,4,1052.0,840,NaN,1000,WN,102,...,0,8,0,NaN,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7450737,2007,12,13,4,1158.0,1200,NaN,1427,DL,670,...,0,16,0,NaN,1,0,0,0,0,0
7451337,2007,12,13,4,653.0,700,NaN,920,DL,1465,...,0,38,0,NaN,1,0,0,0,0,0
7451634,2007,12,13,4,1151.0,1155,NaN,1500,DL,1777,...,0,12,0,NaN,1,0,0,0,0,0
7451702,2007,12,13,4,1153.0,1200,NaN,1507,DL,1844,...,0,12,0,NaN,1,0,0,0,0,0


It looks like all the missing values, except for `CancellationCode` are as a result of the flight being diverted. How true is that? 

In [14]:
flights_NotCancelled[flights_NotCancelled.drop('CancellationCode', axis=1).ArrDelay.isnull()].Diverted.value_counts()

1    17179
Name: Diverted, dtype: int64

In [15]:
flights_NotCancelled[flights_NotCancelled.drop('CancellationCode', axis=1).CRSElapsedTime.isnull()].Diverted.value_counts()

1    727
Name: Diverted, dtype: int64

In [16]:
# clear flights_NotCancelled from memory
del flights_NotCancelled

Check for duplicates

In [17]:
airline_df.duplicated().sum()

27

In [18]:
airline_df[airline_df.duplicated()]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
354402,2007,1,14,7,35.0,35,618.0,605,F9,514,...,37,12,0,NaN,0,0,0,0,0,0
356314,2007,1,21,7,32.0,35,621.0,605,F9,514,...,35,10,0,NaN,0,0,0,16,0,0
4076805,2007,7,23,1,1905.0,1900,2020.0,2035,F9,419,...,6,16,0,NaN,0,0,0,0,0,0
4076807,2007,7,23,1,2136.0,2130,2251.0,2255,F9,419,...,13,11,0,NaN,0,0,0,0,0,0
4076809,2007,7,23,1,619.0,625,931.0,940,F9,222,...,4,12,0,NaN,0,0,0,0,0,0
4076811,2007,7,23,1,1029.0,1025,1630.0,1600,F9,448,...,7,23,0,NaN,0,4,0,26,0,0
4076813,2007,7,23,1,1714.0,1645,1845.0,1905,F9,449,...,6,14,0,NaN,0,0,0,0,0,0
4076815,2007,7,23,1,2004.0,2005,2345.0,2335,F9,237,...,8,16,0,NaN,0,0,0,0,0,0
4076817,2007,7,23,1,610.0,620,723.0,749,F9,378,...,10,9,0,NaN,0,0,0,0,0,0
4076819,2007,7,23,1,2030.0,2030,2346.0,2345,F9,372,...,5,10,0,NaN,0,0,0,0,0,0


In [58]:
airline_df.iloc[4076804:4076824,]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
4076804,2007,7,23,1,1905.0,1900,2020.0,2035,F9,419,N946FR,135.0,155.0,113.0,-15.0,5.0,MDW,DEN,895,6,16,0,NaN,0,0,0,0,0,0
4076805,2007,7,23,1,1905.0,1900,2020.0,2035,F9,419,N946FR,135.0,155.0,113.0,-15.0,5.0,MDW,DEN,895,6,16,0,NaN,0,0,0,0,0,0
4076806,2007,7,23,1,2136.0,2130,2251.0,2255,F9,419,N946FR,135.0,145.0,111.0,-4.0,6.0,DEN,LAX,862,13,11,0,NaN,0,0,0,0,0,0
4076807,2007,7,23,1,2136.0,2130,2251.0,2255,F9,419,N946FR,135.0,145.0,111.0,-4.0,6.0,DEN,LAX,862,13,11,0,NaN,0,0,0,0,0,0
4076808,2007,7,23,1,619.0,625,931.0,940,F9,222,N947FR,132.0,135.0,116.0,-9.0,-6.0,SMF,DEN,910,4,12,0,NaN,0,0,0,0,0,0
4076809,2007,7,23,1,619.0,625,931.0,940,F9,222,N947FR,132.0,135.0,116.0,-9.0,-6.0,SMF,DEN,910,4,12,0,NaN,0,0,0,0,0,0
4076810,2007,7,23,1,1029.0,1025,1630.0,1600,F9,448,N947FR,241.0,215.0,211.0,30.0,4.0,DEN,PHL,1557,7,23,0,NaN,0,4,0,26,0,0
4076811,2007,7,23,1,1029.0,1025,1630.0,1600,F9,448,N947FR,241.0,215.0,211.0,30.0,4.0,DEN,PHL,1557,7,23,0,NaN,0,4,0,26,0,0
4076812,2007,7,23,1,1714.0,1645,1845.0,1905,F9,449,N947FR,211.0,260.0,191.0,-20.0,29.0,PHL,DEN,1557,6,14,0,NaN,0,0,0,0,0,0
4076813,2007,7,23,1,1714.0,1645,1845.0,1905,F9,449,N947FR,211.0,260.0,191.0,-20.0,29.0,PHL,DEN,1557,6,14,0,NaN,0,0,0,0,0,0


In [21]:
airports

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


In [22]:
airports.sample(25)

,iata,airport,city,state,country,lat,long
157,1H0,Creve Coeur,St Louis,MO,USA,38.727520,-90.508304
2041,LBE,Arnold Palmer Regional,Latrobe,PA,USA,40.275940,-79.404797
14,05U,Eureka,Eureka,NV,USA,39.604167,-116.005060
614,77G,Marlette Township,Marlette,MI,USA,43.311830,-83.090914
1175,CQA,Lakefield,Celina,OH,USA,40.484083,-84.560111
1315,DVN,Davenport Municipal,Davenport,IA,USA,41.610278,-90.588325
993,BOS,Gen Edw L Logan Intl,Boston,MA,USA,42.364348,-71.005179
1271,DHT,Dalhart Municipal,Dalhart,TX,USA,36.022585,-102.547278
3163,TYS,McGhee-Tyson,Knoxville,TN,USA,35.812487,-83.992856
962,BIL,Billings Logan Intl,Billings,MT,USA,45.807662,-108.542861


In [23]:
airports.state.unique()

array(['MS', 'TX', 'CO', 'NY', 'FL', 'AL', 'WI', 'OH', 'MO', 'MN', 'IN',
       'NV', 'IL', 'ND', 'MI', 'NE', 'GA', 'DC', 'TN', 'AK', 'ME', 'MA',
       'VT', 'SD', 'NM', 'OK', 'KS', 'KY', 'IA', 'AR', 'LA', 'CA', 'WA',
       'VA', 'AZ', 'PA', 'NJ', 'OR', 'NC', 'UT', 'MT', 'ID', 'CT', 'SC',
       'NH', 'MD', 'DE', 'WV', 'WY', 'PR', 'RI', nan, 'AS', 'CQ', 'GU',
       'HI', 'VI'], dtype=object)

In [24]:
airports.country.unique()

array(['USA', 'Thailand', 'Palau', 'N Mariana Islands',
       'Federated States of Micronesia'], dtype=object)

In [25]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   iata     3376 non-null   object 
 1   airport  3376 non-null   object 
 2   city     3364 non-null   object 
 3   state    3364 non-null   object 
 4   country  3376 non-null   object 
 5   lat      3376 non-null   float64
 6   long     3376 non-null   float64
dtypes: float64(2), object(5)
memory usage: 184.8+ KB


In [26]:
airports.describe()

,lat,long
count,3376.000000,3376.000000
mean,40.036524,-98.621205
std,8.329559,22.869458
min,7.367222,-176.646031
25%,34.688427,-108.761121
50%,39.434449,-93.599425
75%,43.372612,-84.137519
max,71.285448,145.621384


In [27]:
carriers.sample(20)

,Code,Description
1084,RK,Air Afrique
835,ML,Aero Costa Rica
1085,RL,UltrAir
201,APL,Appalachian Flying Service
954,OR,TUI Airlines Nederland B.V.
1016,POQ,Polyot Airlines
926,NZ,Air New Zealand
227,ASP,Asap Air Inc.
646,HQQ,Harlequin Air Corporation
822,MEA,Manhattan Express


In [28]:
carriers.duplicated().sum()

0

In [29]:
carriers.Code.value_counts()

02Q    1
PHL    1
PLA    1
PL     1
PKQ    1
      ..
ENT    1
EMP    1
EME    1
EMA    1
ZYZ    1
Name: Code, Length: 1490, dtype: int64

In [30]:
plane_data.shape

(5029, 9)

In [31]:
plane_data.duplicated().sum()

0

In [32]:
plane_data

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N050AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N051AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N052AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N054AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N055AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5024,N997DL,Corporation,MCDONNELL DOUGLAS AIRCRAFT CO,03/11/1992,MD-88,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1992
5025,N998AT,Corporation,BOEING,01/23/2003,717-200,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2002
5026,N998DL,Corporation,MCDONNELL DOUGLAS CORPORATION,04/02/1992,MD-88,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1992
5027,N999CA,Foreign Corporation,CANADAIR,07/09/2008,CL-600-2B19,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1998


In [104]:
plane_data.describe()

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
count,5029,4480,4480,4480,4480,4480,4480,4480,4480
unique,5029,5,39,2413,167,2,4,7,52
top,N050AA,Corporation,BOEING,12/22/1986,CL-600-2B19,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2001
freq,1,4440,2061,33,403,4479,4440,2883,389


Percentage of NaN values

In [105]:
cols = plane_data.columns

for col in cols:
      # let's find percentage of unique values in each variable
      prop = plane_data[col].isna().sum()/plane_data.shape[0]
      print('{}: {:.3f}%'.format(col, 100*prop))

tailnum: 0.000%
type: 10.917%
manufacturer: 10.917%
issue_date: 10.917%
model: 10.917%
status: 10.917%
aircraft_type: 10.917%
engine_type: 10.917%
year: 10.917%


Let's find out which column can be converted into a categorical one in order to save memory space.

In [69]:
cols = airline_df.columns

for col in cols:
      # let's find percentage of unique values in each variable
      prop = airline_df[col].nunique()/airline_df.shape[0]
      print('{}: {:.2f}%'.format(col, 100*prop))

Year: 0.00%
Month: 0.00%
DayofMonth: 0.00%
DayOfWeek: 0.00%
DepTime: 0.02%
CRSDepTime: 0.02%
ArrTime: 0.02%
CRSArrTime: 0.02%
UniqueCarrier: 0.00%
FlightNum: 0.10%
TailNum: 0.07%
ActualElapsedTime: 0.01%
CRSElapsedTime: 0.01%
AirTime: 0.01%
ArrDelay: 0.02%
DepDelay: 0.02%
Origin: 0.00%
Dest: 0.00%
Distance: 0.02%
TaxiIn: 0.00%
TaxiOut: 0.00%
Cancelled: 0.00%
CancellationCode: 0.00%
Diverted: 0.00%
CarrierDelay: 0.01%
WeatherDelay: 0.01%
NASDelay: 0.01%
SecurityDelay: 0.00%
LateAircraftDelay: 0.01%


In [34]:
cols

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

#### Issues
**`airline_df`**
1. `Year`, `Month`, and `DayofMonth` needs to be combined into one date column ✅
2. Convert `UniqueCarrier`, `FlightNum`, `TailNum`, `Origin`, and `Dest` to categories
3. Drop unnecessary columns [`DayofWeek`, `TailNum`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`]
4. `DepTime`, `CRSDepTime`, `ArrTime`, and `CRSArrTime` need to be represented as datetime dtypes and not numerical data types.
5. Drop duplicate records

**`plane_data`**
1. Drop records with NaN in `type`

### Cleaning data

In [106]:
airline_df_clean = airline_df.copy()
plane_data_clean = plane_data.copy()

##### `Year`, `Month`, and `DayofMonth` needs to be combined into one date column in **`airline_df`**

**Define**
* Rename `DayofMonth` to `Day`
* Use pd.to_datetime() to combine `Year`, `Month` and `Day` into `Date` column.
* Bring forward the `Date` column.
* Drop `Year`, `Month` and `Day`

**Code**

In [87]:
# rename DayofMonth
airline_df_clean.rename(columns={'DayofMonth':'Day'}, inplace=True)

In [88]:
# combine 'Year', 'Month' and 'Day'
date = pd.to_datetime(airline_df_clean[['Year','Month','Day']])

# insert date as first column, 'Date'
airline_df_clean.insert(0, 'Date', date)

# drop Year, Month and Day
airline_df_clean.drop(['Year','Month','Day'], axis=1, inplace=True)

In [89]:
airline_df_clean.head()

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2007-01-01,1,1232.0,1225,1341.0,1340,WN,2891,N351,69.0,75.0,54.0,1.0,7.0,SMF,ONT,389,4,11,0,NaN,0,0,0,0,0,0
1,2007-01-01,1,1918.0,1905,2043.0,2035,WN,462,N370,85.0,90.0,74.0,8.0,13.0,SMF,PDX,479,5,6,0,NaN,0,0,0,0,0,0
2,2007-01-01,1,2206.0,2130,2334.0,2300,WN,1229,N685,88.0,90.0,73.0,34.0,36.0,SMF,PDX,479,6,9,0,NaN,0,3,0,0,0,31
3,2007-01-01,1,1230.0,1200,1356.0,1330,WN,1355,N364,86.0,90.0,75.0,26.0,30.0,SMF,PDX,479,3,8,0,NaN,0,23,0,0,0,3
4,2007-01-01,1,831.0,830,957.0,1000,WN,2278,N480,86.0,90.0,74.0,-3.0,1.0,SMF,PDX,479,3,9,0,NaN,0,0,0,0,0,0


In [90]:
airline_df_clean.Date.dtype

dtype('<M8[ns]')

##### Convert `UniqueCarrier`, `FlightNum`, `TailNum`, `Origin`, and `Dest` to categories in `airline_df`
**Describe**
* Get list of columns to convert
* Iterate through leist and convert each column to a cat type

**Code**

In [91]:
cat_cols = ['UniqueCarrier', 'FlightNum', 'TailNum', 'Origin', 'Dest']

for col in cat_cols:
      airline_df_clean[col] = airline_df_clean[col].astype('category')

**Test**

In [92]:
airline_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7453215 entries, 0 to 7453214
Data columns (total 27 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Date               datetime64[ns]
 1   DayOfWeek          int64         
 2   DepTime            float64       
 3   CRSDepTime         int64         
 4   ArrTime            float64       
 5   CRSArrTime         int64         
 6   UniqueCarrier      category      
 7   FlightNum          category      
 8   TailNum            category      
 9   ActualElapsedTime  float64       
 10  CRSElapsedTime     float64       
 11  AirTime            float64       
 12  ArrDelay           float64       
 13  DepDelay           float64       
 14  Origin             category      
 15  Dest               category      
 16  Distance           int64         
 17  TaxiIn             int64         
 18  TaxiOut            int64         
 19  Cancelled          int64         
 20  CancellationCode   objec

##### Drop unnecessary columns in `airline_df`
**Describe**
* Get list of columns with redundant information
* Drop such columns

**Code**

In [98]:
dropCols = ['DayOfWeek','TailNum', 'ActualElapsedTime',
        'CRSElapsedTime', 'AirTime']

airline_df_clean.drop(dropCols, axis=1, inplace=True)

**Test**

In [99]:
airline_df_clean.columns

Index(['Date', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime',
       'UniqueCarrier', 'FlightNum', 'ArrDelay', 'DepDelay', 'Origin', 'Dest',
       'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode',
       'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay'],
      dtype='object')

##### Drop duplicate records in `airline_df`
**Describe**
* Drop duplicates in airline_df_clean

**Code**

In [101]:
airline_df_clean.drop_duplicates(inplace=True)

**Test**

In [102]:
airline_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7453188 entries, 0 to 7453214
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Date               datetime64[ns]
 1   DepTime            float64       
 2   CRSDepTime         int64         
 3   ArrTime            float64       
 4   CRSArrTime         int64         
 5   UniqueCarrier      category      
 6   FlightNum          category      
 7   ArrDelay           float64       
 8   DepDelay           float64       
 9   Origin             category      
 10  Dest               category      
 11  Distance           int64         
 12  TaxiIn             int64         
 13  TaxiOut            int64         
 14  Cancelled          int64         
 15  CancellationCode   object        
 16  Diverted           int64         
 17  CarrierDelay       int64         
 18  WeatherDelay       int64         
 19  NASDelay           int64         
 20  SecurityDelay      int64

##### Drop records with NaN in `type` in `plane_data`
**Describe**
* Get the subset of `plane_data_clean` that are NaN
* Drop that subset from the main df

**Code**

In [107]:
plane_data_clean[plane_data_clean['type'].isna()]

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N050AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N051AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N052AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N054AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N055AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4717,N9331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4720,N9336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4721,N9337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4796,N942AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
plane_data_clean.dropna(subset='type', inplace=True)

**Test**

In [109]:
plane_data_clean[plane_data_clean['type'].isna()]

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year


### Storing Data

In [110]:
airline_df_clean.to_csv('airline_clean.csv', index=False)
plane_data_clean.to_csv('plane_clean.csv', index=False)